# PyCryption Research Tutorial

This notebook serves a technical introduction, demonstration, and collaboration baseline for this repository, PyCryption. It values 'fire-and-forget' systems to allow you to write, and test custom encryption systems.

The true way to measure performance would be analyze mathematical complexity, but this is a good baseline for understanding the performance of your system, especially in production, and prototyping.


---

## 0.0 - Getting Started: Building a Basic Algorithm

In this section we'll exploring how to rapidly test new encryption algorithms without writing tedious harnesses for testing.

### 0.1 Understand the Mental Model

1. **Define your encryption algorithm into a class with two endpoint functions**: `encrypt`, and `decrypt`, that can accept data, and the algorithm context.
2. **Perform the `quick_test`** to ensure successful dataflow.
3. Register other models and algorithms to compare.
4. Refine and/or repeat.

### 0.2 Encryption Helpers and Algorithm Context

An incredibly powerful component, `AlgorithmContext` is a communication interface to your algorithm from the cryptography components you add to your algorithm.


In [14]:
from lib.notebook import (
    algorithm,
    with_key,
    generate_key,
    AlgorithmContext,
    ComposerSession,
    ReportBuilder,
    quick_test,
)

# Initialize report builder for styled output
report = ReportBuilder()

# -----------------------------------------------------------------------------
# Prototype Algorithm Development
# -----------------------------------------------------------------------------
# Use decorators for logistics (key injection, context, metrics).
# Write your own experimental crypto logic in the methods.

KEY = generate_key(32)


@algorithm("XOR-Prototype")
@with_key(KEY)
class XORPrototype:
    """
    Simple, insecure XOR-based prototype for testing the framework.
    """

    def encrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # Prototype: simple repeating-key XOR
        key = ctx.key
        return bytes(b ^ key[i % len(key)] for i, b in enumerate(data))

    def decrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # XOR is symmetric
        key = ctx.key
        return bytes(b ^ key[i % len(key)] for i, b in enumerate(data))


# Verify the prototype works
quick_test(XORPrototype())

Testing: XOR-Prototype
Input: b'Hello, PyCryption!'
----------------------------------------
Encrypt: <AlgorithmResult: 18 bytes, 0.029ms>
Decrypt: <AlgorithmResult: 18 bytes, 0.016ms>
Round-trip successful!


In [15]:
# -----------------------------------------------------------------------------
# Composer Session - Benchmark Prototype vs Production Algorithms
# -----------------------------------------------------------------------------

from lib.notebook import wrap_aes256gcm

# Create session and register algorithms
session = ComposerSession()
session.register(XORPrototype())
session.register(wrap_aes256gcm(KEY, name="AES-256-GCM"))

report.info(f"Registered: {session.list_algorithms()}")

# Test all registered algorithms
report.heading("Round-trip Tests", level=2)
report.test_results(session.test_all())


ℹ Registered: ['XOR-Prototype', 'AES-256-GCM']

Round-trip Tests

     Round-trip Tests     
┏━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Algorithm     ┃ Status ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ XOR-Prototype │ ✓ PASS │
│ AES-256-GCM   │ ✓ PASS │
└───────────────┴────────┘

In [16]:
# -----------------------------------------------------------------------------
# Benchmark Prototype Performance
# -----------------------------------------------------------------------------

report.heading("Prototype Benchmark (10KB, 50 iterations)", level=2)
report.comparison_table(session.compare(data_size=10_000, iterations=50))


Prototype Benchmark (10KB, 50 iterations)

                       Algorithm Comparison                        
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Algorithm     ┃ Encrypt (ms) ┃ Decrypt (ms) ┃ Throughput (MB/s) ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ AES-256-GCM   │ 0.04         │ 0.036        │ 248.51            │
│ XOR-Prototype │ 1.789        │ 1.796        │ 5.59              │
└───────────────┴──────────────┴──────────────┴───────────────────┘

In [17]:
# -----------------------------------------------------------------------------
# Detailed Benchmarks - Scaling Analysis
# -----------------------------------------------------------------------------

report.heading("Scaling Analysis", level=2)
report.benchmark_table(
    session.benchmark_all(data_sizes=[100, 1_000, 10_000, 100_000, 100_000], iterations=20)
)


Scaling Analysis

                     XOR-Prototype                      
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Size      ┃ Encrypt (ms) ┃ Decrypt (ms) ┃ Throughput ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 100 B     │        0.023 │        0.022 │  4.26 MB/s │
│ 1,000 B   │        0.217 │        0.223 │  4.61 MB/s │
│ 10,000 B  │        1.743 │        1.809 │  5.74 MB/s │
│ 100,000 B │       10.507 │        9.949 │  9.52 MB/s │
│ 100,000 B │        9.867 │        9.967 │ 10.13 MB/s │
└───────────┴──────────────┴──────────────┴────────────┘

                       AES-256-GCM                        
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Size      ┃ Encrypt (ms) ┃ Decrypt (ms) ┃   Throughput ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ 100 B     │        0.019 │        0.014 │    5.14 MB/s │
│ 1,000 B   │        0.011 │         0.01 │    93.9 MB/s │
│ 10,000 B  │        0.011 │         0.01 │  921.66 MB/s │
│ 100,000 B │        0.039 │        0.036 │ 2554.28 MB/s │
│ 100,000 B │        0.043 │        0.041 │ 2341.92 MB/s │
└───────────┴──────────────┴──────────────┴──────────────┘

In [18]:
# -----------------------------------------------------------------------------
# Session Metrics - Aggregated Statistics
# -----------------------------------------------------------------------------

report.heading("Session Report", level=2)
report.session_report(session.report())


Session Report

                                      Session Report                                       
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Algorithm     ┃ Operations ┃ Avg Encrypt (ms) ┃ Avg Decrypt (ms) ┃ Total Bytes ┃ Errors ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━┩
│ XOR-Prototype │ 1E / 1D    │ 0.032            │ 0.015            │ 36          │ -      │
│ AES-256-GCM   │ 1E / 1D    │ 0.242            │ 0.056            │ 52          │ -      │
└───────────────┴────────────┴──────────────────┴──────────────────┴─────────────┴────────┘

---

## 1.0: Adding Cryptographic Capabilities

Beyond a simple byte emitter in the last example, we can easily add new capabilities. Let's take a look at the [Salsa20](https://en.wikipedia.org/wiki/Salsa20) algorithm which was succeeded by its descendent, ChaCha in 2008. Though it's dated, it still uses much of the same IO of modern ciphers.


In [19]:
import hashlib
from Crypto.Cipher import Salsa20
from lib.notebook import with_metrics


def salsa_kdf(key: bytes, salt: bytes) -> bytes:
    return hashlib.pbkdf2_hmac('sha256', key, salt, 100000, dklen=32)


@algorithm("Salsa20-Prototype")
@with_key(KEY)
@with_metrics()
class Salsa20Prototype:
    """
    A classic encryption cipher, Salsa20 was shelved in 2008.
    """

    def encrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # Register KDF and salt, derive key, cache result
        ctx.set_kdf("salsa-kdf", salsa_kdf)
        ctx.set_salt("salsa-salt")
        derived = ctx.derive("salsa-kdf", "salsa-salt", cache_as="salsa-derived")


        cipher = Salsa20.new(derived)
        ctx.set_nonce("salsa-nonce", cipher.nonce)
        return cipher.encrypt(data)

    def decrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # Retrieve cached materials from registry
        derived = ctx.get_derived_key("salsa-derived")
        nonce = ctx.get_nonce("salsa-nonce")
        cipher = Salsa20.new(derived, nonce)
        return cipher.decrypt(data)


# Verify the prototype works
quick_test(Salsa20Prototype())
session = ComposerSession()
# load the Salsa20 cipher with the password KDF
session.register(Salsa20Prototype(), "Salsa20-Prototype")
# load the AES-256-GCM cipher with the password KDF
session.register(wrap_aes256gcm(KEY))

Testing: Salsa20-Prototype
Input: b'Hello, PyCryption!'
----------------------------------------
Encrypt: <AlgorithmResult: 18 bytes, 36.71ms>
Decrypt: <AlgorithmResult: 18 bytes, 0.027ms>
Round-trip successful!


In [20]:
report.heading("Scaling Analysis", level=2)
report.benchmark_table(
    session.benchmark_all(data_sizes=[100, 1_000, 10_000, 100_000, 100_000], iterations=20)
)

Scaling Analysis

                   Salsa20-Prototype                    
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Size      ┃ Encrypt (ms) ┃ Decrypt (ms) ┃ Throughput ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 100 B     │       30.302 │        0.016 │   0.0 MB/s │
│ 1,000 B   │       31.146 │        0.018 │  0.03 MB/s │
│ 10,000 B  │       23.213 │        0.029 │  0.43 MB/s │
│ 100,000 B │       21.305 │        0.204 │  4.69 MB/s │
│ 100,000 B │       23.621 │        0.226 │  4.23 MB/s │
└───────────┴──────────────┴──────────────┴────────────┘

                       AES-256-GCM                        
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Size      ┃ Encrypt (ms) ┃ Decrypt (ms) ┃   Throughput ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ 100 B     │        0.014 │        0.011 │    7.33 MB/s │
│ 1,000 B   │        0.008 │        0.007 │   125.0 MB/s │
│ 10,000 B  │         0.01 │        0.009 │ 1047.12 MB/s │
│ 100,000 B │         0.03 │        0.029 │ 3300.33 MB/s │
│ 100,000 B │        0.029 │        0.028 │ 3442.34 MB/s │
└───────────┴──────────────┴──────────────┴──────────────┘

## 1.1 *Analyzing our Results*
